In [16]:
import numpy as np
import scipy.special as S
import matplotlib.pyplot as plt
class neuralNetwork:
#初始化神经网络,构造函数
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        #设置每个输入、隐藏、输出层中的节点数
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        #链接权重矩阵，wih和who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = np.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))
        
        #学习率
        self.lr = learningrate
        
        #创建激活函数（函数的另一种定义方法,这样以后可以直接调用）
        self.activation_function = lambda x: S.expit(x)
        
        pass
    
#训练神经网络
    def train(self, inputs_list, targets_list):
        #将输入列表转换成二维数组
        inputs = np.array(inputs_list, ndmin = 2).T
        targets = np.array(targets_list, ndmin = 2).T
        
        #将输入信号计算到隐藏层
        hidden_inputs = np.dot(self.wih, inputs)
        #计算隐藏层中输出的信号(使用激活函数计算)
        hidden_outputs = self.activation_function(hidden_inputs)
        #将传输的信号计算到输出层
        final_inputs = np.dot(self.who, hidden_outputs)
        #计算输出层中输出的信号（使用激活函数）
        final_outputs = self.activation_function(final_inputs)
        
        #计算输出层的误差：（target - actual）(预期目标输出值-实际计算得到的输出值)
        output_errors = targets - final_outputs
        #隐藏层的误差：是输出层误差按权重分割，在隐藏节点上重新组合
        hidden_errors = np.dot(self.who.T, output_errors)
        
        #反向传播，更新各层权重
        #更新隐层和输出层之间的权重
        self.who += self.lr*np.dot((output_errors*final_outputs*(1.0 - final_outputs)), np.transpose(hidden_outputs))
        #更新输入层和隐藏层之间的权重
        self.wih += self.lr*np.dot((hidden_errors*hidden_outputs*(1.0 - hidden_outputs)), np.transpose(inputs))
        #pass一般用于占位置，定义一个空函数程序会报错，当没有想好函数的内容可以用pass填充，使得程序正常运行
        pass
    
#查询神经网络：接受神经网络的输入，返回神经网络的输出
    def query(self, inputs_list):
        #将输入列表转换成二维数组
        inputs = np.array(inputs_list, ndmin = 2).T
        
        #将输入信号计算到隐藏层
        hidden_inputs = np.dot(self.wih, inputs)
        #将信号从隐藏层输出
        hidden_outputs = self.activation_function(hidden_inputs)
        #将信号引入到输出层
        final_inputs = np.dot(self.who, hidden_outputs)
        #将信号从输出层输出
        final_outputs = self.activation_function(final_inputs)
        
        #返回输出层的输出值
        return final_outputs

### 参数设置进行实际训练测试

In [17]:
input_nodes = 784
hidden_nodes = 200
output_nodes = 10
learning_rate = 0.3
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

 #将mnist的训练数据CSV文件加载到一个列表中
training_data_file = open("mnist_dataset/mnist_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [20]:
%%time
#训练神经网络
#epochs是用于训练数据集的次数，有些人把训练一次叫做一个世代，
epochs = 5
for e in range(epochs):
    for record in training_data_list:
        all_values = record.split(',')
        inputs = (np.asfarray(all_values[1:]) / 255.0*0.99) + 0.01
        targets = np.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    pass

Wall time: 6min 42s


In [21]:
#将mnist测试数据的CSV文件加载到一个列表中
test_data_file = open("mnist_dataset/mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

#scorecard用于记录训练好的神经网络在测试时的表现，初始为一个空列表
scorecard = []

In [23]:
%%time
#测试神经网络
#检测测试数据集中的所有数据
for record in test_data_list:
    all_values = record.split(',')
    correct_label = int(all_values[0])
    inputs = (np.asfarray(all_values[1:]) / 255.0*0.99) + 0.01
    outputs = n.query(inputs)
    outputs_max_label = np.argmax(outputs)
    if (outputs_max_label == correct_label):
        scorecard.append(1)
    else:
        scorecard.append(0)
        pass
    pass

Wall time: 1.86 s


In [ ]:
#计算得分
scorecard_array = np.asarray(scorecard)
print("performace = ", scorecard_array.sum() / scorecard_array.size)